In [85]:
%load_ext autoreload
%autoreload 2
import os
from pipeline import utils
from model import drivenet
import tensorflow as tf
from model.driving_data import Dataset

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [86]:
os.getcwd()

'/Users/daniel/Dropbox/udacity_challenge2'

In [92]:
model_dict = {

    'driveNet': drivenet.DriveNet

}



In [93]:
model_configs = utils.from_recipe()

{'MODEL_TITLE': 'driveNet', 'CHANNEL': 3, 'HEIGHT': 66, 'BATCH_SIZE': 100, 'NUM_ITER': 100000, 'WIDTH': 200, 'MODEL_FILE': 'drivenet'}


In [101]:
for config in model_configs:

    # config
    NUM_ITER = config["NUM_ITER"]
    BATCH_SIZE = config["BATCH_SIZE"]
    MODEL_TITLE = config["MODEL_TITLE"]
    MODEL_FILE = config["MODEL_FILE"]
    SUMMARY_DIR = 'save/log/' + MODEL_TITLE
    WIDTH = config["WIDTH"]
    HEIGHT = config["HEIGHT"]
    CHANNEL = config["CHANNEL"]

    if not os.path.exists(SUMMARY_DIR):
        os.makedirs(SUMMARY_DIR)
    # get session
    sess = tf.InteractiveSession()

    # setupt dataset
    driving_data = Dataset(DATA_DIR='rawdata/driving_dataset', file_name='data.txt', width=WIDTH, height=HEIGHT)

    # setup model
    dnn = model_dict[MODEL_TITLE](width=WIDTH, height=HEIGHT, channel=CHANNEL)
    dnn.inference()

    with tf.name_scope('loss'):
        loss = tf.reduce_mean(tf.square(tf.sub(dnn.y_, dnn.y)))
        tf.scalar_summary('mse', loss)

    # set up optimizer
    train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
    sess.run(tf.initialize_all_variables())

    # summary statistics
    merged = tf.merge_all_summaries()
    train_writer = tf.train.SummaryWriter(SUMMARY_DIR + '/train', sess.graph)
    test_writer = tf.train.SummaryWriter(SUMMARY_DIR + '/test')

    # saver
    saver = tf.train.Saver()

    for i in range(NUM_ITER):
        xs, ys = driving_data.LoadTrainBatch(BATCH_SIZE)
        print(xs)
        print(ys)
        if i % 10 == 0:
            xs, ys = driving_data.LoadValBatch(BATCH_SIZE)
            summary, mse = sess.run([merged, loss], feed_dict={dnn.x: xs, dnn.y_: ys, dnn.keep_prob: 0.8})
            test_writer.add_summary(summary, i)
        elif i % 100 == 99:  # Record execution stats
            run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
            run_metadata = tf.RunMetadata()
            summary, _ = sess.run([merged, train_step],
                                  feed_dict={dnn.x: xs, dnn.y_: ys, dnn.keep_prob: 0.8},
                                  options=run_options,
                                  run_metadata=run_metadata)
            train_writer.add_run_metadata(run_metadata, 'step%03d' % i)
            train_writer.add_summary(summary, i)
            print('Adding run metadata for', i)
        else:
            summary, _ = sess.run([merged, train_step], feed_dict={dnn.x: xs, dnn.y_: ys, dnn.keep_prob: 0.8})
            train_writer.add_summary(summary, i)

        if i % 100 == 0:
            if not os.path.exists(LOGDIR):
                os.makedirs(LOGDIR)
            #
            checkpoint_path = os.path.join(LOGDIR, MODEL_TITLE + ".ckpt")
            filename = saver.save(sess, checkpoint_path)
            print("Model saved in file: %s" % filename)

            json_data = {"iter": i, "mse": mse}
            utils.append(MODEL_TITLE, json_data)

    train_writer.close()
    test_writer.close()

Exception ignored in: <bound method BaseSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x133208780>>
Traceback (most recent call last):
  File "/Users/daniel/anaconda3/envs/tensorflow/lib/python3.5/site-packages/tensorflow/python/client/session.py", line 522, in __del__
    self.close()
  File "/Users/daniel/anaconda3/envs/tensorflow/lib/python3.5/site-packages/tensorflow/python/client/session.py", line 1262, in close
    self._default_session.__exit__(None, None, None)
  File "/Users/daniel/anaconda3/envs/tensorflow/lib/python3.5/contextlib.py", line 66, in __exit__
    next(self.gen)
  File "/Users/daniel/anaconda3/envs/tensorflow/lib/python3.5/site-packages/tensorflow/python/framework/ops.py", line 3536, in get_controller
    % type(default))
AssertionError: Nesting violated for default stack of <class 'weakref'> objects


[array([[[ 0.6       ,  0.52156863,  0.38823529],
        [ 0.57647059,  0.54117647,  0.33333333],
        [ 0.54901961,  0.53333333,  0.29019608],
        ..., 
        [ 0.45490196,  0.48627451,  0.2745098 ],
        [ 0.42745098,  0.43921569,  0.27843137],
        [ 0.48627451,  0.49019608,  0.32156863]],

       [[ 0.55294118,  0.46666667,  0.36078431],
        [ 0.57647059,  0.53333333,  0.29411765],
        [ 0.57254902,  0.54901961,  0.27843137],
        ..., 
        [ 0.37647059,  0.41568627,  0.30980392],
        [ 0.4       ,  0.43137255,  0.28627451],
        [ 0.48235294,  0.50980392,  0.27843137]],

       [[ 0.56862745,  0.47058824,  0.30980392],
        [ 0.60392157,  0.54117647,  0.26666667],
        [ 0.61176471,  0.57647059,  0.26666667],
        ..., 
        [ 0.43137255,  0.47058824,  0.38823529],
        [ 0.40392157,  0.43921569,  0.31372549],
        [ 0.43137255,  0.45882353,  0.30196078]],

       ..., 
       [[ 0.4745098 ,  0.41960784,  0.41568627],
       

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_12' with dtype float
	 [[Node: Placeholder_12 = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]
Caused by op 'Placeholder_12', defined at:
  File "/Users/daniel/anaconda3/envs/tensorflow/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/daniel/anaconda3/envs/tensorflow/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/daniel/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/daniel/anaconda3/envs/tensorflow/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/daniel/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/Users/daniel